In [8]:
import pandas as pd
import numpy as np
import os
import sys
import json
import requests
import gc
from tqdm import tqdm

from google.cloud import bigquery
from google.cloud.bigquery import job
from datetime import date, timedelta, datetime

PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)

#  클라이언트 설정



In [15]:
query = """
    with click1 as 
  (select event_dttm, event_dt, user_id
  FROM `ballosodeuk.dw.fact_airbridge_event`, unnest(event_detail)
  where 1=1
    and event_dt between "2024-12-30" and {end_date}
    and event_category in ('c__shopping_home__quick__btn (App)'))

,click2 as 
  (select event_dttm, event_dt, user_id
  FROM `ballosodeuk.dw.fact_airbridge_event`, unnest(event_detail)
  where 1=1
    and event_dt between "2024-12-30" and {end_date}
    and event_category in ('c__all_income_s_money__to_exchange__btn (App)')
    )


,user_param as (
select distinct event_dt, user_id
from
  (select distinct event_dt, user_id from click1
  union all
  select distinct event_dt, user_id from click2
  ))

select a.user_id, event_dt, current_cash
from user_param a
left join ballosodeuk.dw.dim_airbridge_member b on a.user_id = b.user_id
order by user_id, event_dt

"""



In [24]:
# yester_day = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
# formatted_query = query.format(
#     end_date=f"'{yester_day}'"
# )

# df = bq.query(formatted_query).to_dataframe()
df.to_parquet(f"./data/click_param_{yester_day}.parquet")
df = pd.read_parquet(f"./data/click_param_{yester_day}.parquet")

In [25]:
df = df.query("user_id.notnull()")
df.iloc[:,-1] = df.iloc[:,-1].fillna(0).astype(int)
df['event_dt'] = pd.to_datetime(df.event_dt.astype(str))

<ipython-input-25-90d58f8dbcc6>:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,-1] = df.iloc[:,-1].fillna(0).astype(int)


In [45]:
df.sort_values(by=['user_id','event_dt'],inplace=True)
df['days_between_visits'] = df.groupby('user_id').event_dt.diff().dt.days


,user_id,event_dt,current_cash,days_between_visits
31,000087ef-6929-4532-85be-33661b2082d5,2025-01-06,25905,NaN
32,000087ef-6929-4532-85be-33661b2082d5,2025-01-10,25905,4.0
33,000087ef-6929-4532-85be-33661b2082d5,2025-01-13,25905,3.0
34,000087ef-6929-4532-85be-33661b2082d5,2025-01-16,25905,3.0
35,000087ef-6929-4532-85be-33661b2082d5,2025-01-17,25905,1.0


In [40]:
# 더 효율적인 방법으로 일별 커버리지 계산
def calculate_daily_coverage(df):
    # 전체 유니크 유저 수 계산
    total_unique_users = df['user_id'].nunique()
    
    # 날짜별 커버리지 계산 (vectorized operation 사용)
    daily_coverage = (df.groupby('event_dt')['user_id']
                     .nunique()
                     .reset_index()
                     .rename(columns={'user_id': 'daily_users'}))
    
    # 커버리지 비율 계산
    daily_coverage['coverage_rate'] = (daily_coverage['daily_users'] / total_unique_users * 100)
    
    return daily_coverage[['event_dt', 'coverage_rate','daily_users']]

# 결과 출력
daily_coverage = calculate_daily_coverage(df)
print("일자별 전체 유니크 유저 포함률:")
print(daily_coverage.set_index('event_dt').sort_index())
print(f"\n평균 포함률: {daily_coverage['coverage_rate'].mean():.1f}%")

일자별 전체 유니크 유저 포함률:
            coverage_rate  daily_users
event_dt                              
2024-12-30      32.707837        52169
2024-12-31      29.477116        47016
2025-01-01      22.643260        36116
2025-01-02      20.839498        33239
2025-01-03      17.642006        28139
2025-01-04      18.492163        29495
2025-01-05      16.682132        26608
2025-01-06      16.972414        27071
2025-01-07      15.588715        24864
2025-01-08      14.265204        22753
2025-01-09      14.079624        22457
2025-01-10      13.101567        20897
2025-01-11      13.683386        21825
2025-01-12      13.549843        21612
2025-01-13      14.109718        22505
2025-01-14      13.675862        21813
2025-01-15      12.784953        20392
2025-01-16      12.937304        20635
2025-01-17      12.004389        19147
2025-01-18      12.496552        19932
2025-01-19      12.073354        19257
2025-01-20      12.499687        19937
2025-01-21      12.080878        19269
2025-0

In [48]:
user_visit_gaps = df.groupby('user_id')['days_between_visits'].mean()
user_visit_gaps.describe()
gap_dist = user_visit_gaps.value_counts().sort_index()
gap_dist


1.000000     7424
1.028571      370
1.029412      130
1.030303       39
1.031250       24
1.032258       11
1.033333       15
1.034483       14
1.035714       13
1.037037        6
1.038462       15
1.040000        9
1.041667        4
1.043478       12
1.045455        9
1.047619        6
1.050000        7
1.052632        4
1.055556        6
1.058824      373
1.060606      132
1.062500       71
1.064516       41
1.066667       25
1.068966       27
1.071429       21
1.074074       16
1.076923       21
1.080000       13
1.083333       28
1.086957        7
1.090909      385
1.093750      138
1.095238        7
1.096774       71
1.100000       62
1.103448       28
1.105263        9
1.107143       24
1.111111       44
1.115385       21
1.117647        7
1.120000       12
1.125000      411
1.129032      164
1.130435       20
1.133333       77
1.136364       13
1.137931       36
1.142857       88
1.148148       21
1.150000        7
1.153846       29
1.157895        7
1.160000       17
1.161290  

In [31]:
df_g = df.groupby('event_dt').agg({'optimal_coupon':'sum','left_cash':'sum','user_id':'count','current_cash':'sum'})
df_g['left_cash_rate'] = df_g['left_cash'] / df_g['current_cash']
df_g

KeyError: "Column(s) ['left_cash', 'optimal_coupon'] do not exist"

In [46]:
len(df.user_id.unique())
len(df[['user_id']].drop_duplicates())

159500

In [39]:
import pandas as pd
import numpy as np

def find_optimal_coupon(cash, coupon_price):
    coupon_array = np.array(coupon_price)
    tg_buy_idx = np.where((cash - coupon_array) >= 0, cash - coupon_array, np.inf).argmin()
    return coupon_array[tg_buy_idx], cash - coupon_array[tg_buy_idx]

def simulate_daily_purchases(user_data, days, coupon_price, min_coupon):
    results = []
    
    # 초기 활성 사용자 설정
    active_users = user_data.copy()
    active_users['cash'] = active_users['current_cash']  # 초기 잔액 설정
    
    for day in range(1, days + 1):
        # 당일 방문 가능한 사용자 (cash가 min_coupon 이상인 사용자)
        today_users = active_users[active_users['cash'] >= min_coupon].copy()
        
        if len(today_users) > 0:
            # 쿠폰 구매 및 잔액 계산
            optimal_coupons = []
            left_cash = []
            
            for cash in today_users['cash']:
                coupon, remaining = find_optimal_coupon(cash, coupon_price)
                optimal_coupons.append(coupon)
                left_cash.append(remaining)
            
            # 사용자 잔액 업데이트
            today_users['cash'] = left_cash
            active_users.loc[today_users.index, 'cash'] = left_cash
            
            # 일일 결과 저장
            results.append({
                'day': day,
                'total_coupon_amount': sum(optimal_coupons),
                'total_remaining_cash': active_users['cash'].sum(),
                'active_users': len(active_users),
                'today_visitors': len(today_users)
            })
        else:
            # 더 이상 방문 가능한 사용자가 없는 경우
            results.append({
                'day': day,
                'total_coupon_amount': 0,
                'total_remaining_cash': active_users['cash'].sum(),
                'active_users': len(active_users),
                'today_visitors': 0
            })
            # 모든 사용자의 잔액이 min_coupon 미만이면 시뮬레이션 종료
            break
    
    return pd.DataFrame(results)

# 시뮬레이션 실행
days = 100
coupon_price = [400, 1200, 2000, 4000, 12000]
min_coupon = min(coupon_price)

df__ = df[['user_id', 'current_cash']].drop_duplicates()
results_df = simulate_daily_purchases(df__[['user_id', 'current_cash']], days, coupon_price, min_coupon)

# 결과 출력
print("\n일별 시뮬레이션 결과:")
print(results_df.to_string(index=False))

# 추가 분석
print(f"\n시뮬레이션 종료일: {len(results_df)}일")
print(f"최종 남은 총 잔액: {results_df.iloc[-1]['total_remaining_cash']:,.0f}원")
print(f"총 쿠폰 구매액: {results_df['total_coupon_amount'].sum():,.0f}원")


일별 시뮬레이션 결과:
 day  total_coupon_amount  total_remaining_cash  active_users  today_visitors
   1           1149719200            1449125840        159500          149648
   2            723660000             725465840        159500          136146
   3            390842800             334623040        159500          107250
   4            184463200             150159840        159500           70238
   5             75768400              74391440        159500           39173
   6             28078400              46313040        159500           18593
   7              9541600              36771440        159500            7519
   8              3216800              33554640        159500            2599
   9              1176400              32378240        159500             787
  10               532000              31846240        159500             256
  11               342400              31503840        159500              99
  12               221600              31282240   

In [41]:
import pandas as pd
import numpy as np

def find_optimal_coupon(cash, coupon_price):
   coupon_array = np.array(coupon_price)  
   tg_buy_idx = np.where((cash - coupon_array) >= 0, cash - coupon_array, np.inf).argmin()
   return coupon_array[tg_buy_idx], cash - coupon_array[tg_buy_idx]

def simulate_daily_purchases(user_data, days, coupon_price, min_coupon, daily_cap):
   results = []
   
   # 초기 활성 사용자 설정
   active_users = user_data.copy()
   active_users['cash'] = active_users['current_cash']  # 초기 잔액 설정
   
   for day in range(1, days + 1):
       # 당일 방문 가능한 사용자 (cash가 min_coupon 이상인 사용자)
       potential_users = active_users[active_users['cash'] >= min_coupon]
       
       if len(potential_users) > 0:
           # 일일 방문자 수를 daily_cap으로 제한
           n_visitors = min(len(potential_users), daily_cap)
           # 랜덤 샘플링으로 today_users 선택
           today_users = potential_users.sample(n=n_visitors).copy()
           
           # 쿠폰 구매 및 잔액 계산
           optimal_coupons = []
           left_cash = []
           
           for cash in today_users['cash']:
               coupon, remaining = find_optimal_coupon(cash, coupon_price)
               optimal_coupons.append(coupon)
               left_cash.append(remaining)
           
           # 사용자 잔액 업데이트
           today_users['cash'] = left_cash
           active_users.loc[today_users.index, 'cash'] = left_cash
           
           # 일일 결과 저장
           results.append({
               'day': day,
               'total_coupon_amount': sum(optimal_coupons),
               'total_remaining_cash': active_users['cash'].sum(),
               'active_users': len(active_users),
               'today_visitors': len(today_users),
               'potential_visitors': len(potential_users)
           })
       else:
           # 더 이상 방문 가능한 사용자가 없는 경우
           results.append({
               'day': day,
               'total_coupon_amount': 0,
               'total_remaining_cash': active_users['cash'].sum(),
               'active_users': len(active_users),
               'today_visitors': 0,
               'potential_visitors': 0
           })
           # 모든 사용자의 잔액이 min_coupon 미만이면 시뮬레이션 종료
           break
   
   return pd.DataFrame(results)

# 시뮬레이션 실행
days = 100
coupon_price = [400, 1200, 2000, 4000, 12000]
min_coupon = min(coupon_price)
daily_cap = 22000

df__ = df[['user_id', 'current_cash']].drop_duplicates()
results_df = simulate_daily_purchases(df__[['user_id', 'current_cash']], days, coupon_price, min_coupon, daily_cap)

# 결과 출력
print("\n일별 시뮬레이션 결과:")
print(results_df.to_string(index=False))

# 추가 분석
print(f"\n시뮬레이션 종료일: {len(results_df)}일")
print(f"최종 남은 총 잔액: {results_df.iloc[-1]['total_remaining_cash']:,.0f}원")
print(f"총 쿠폰 구매액: {results_df['total_coupon_amount'].sum():,.0f}원")

pd.set_option('display.max_rows', None) 
pd.DataFrame(results_df)




일별 시뮬레이션 결과:
 day  total_coupon_amount  total_remaining_cash  active_users  today_visitors  potential_visitors
   1            168924000            2429921040        159500           22000              149648
   2            161690800            2268230240        159500           22000              147644
   3            156117600            2112112640        159500           22000              145222
   4            149026800            1963085840        159500           22000              142484
   5            143897200            1819188640        159500           22000              139453
   6            138155600            1681033040        159500           22000              136138
   7            131958800            1549074240        159500           22000              132439
   8            126724400            1422349840        159500           22000              128444
   9            121508800            1300841040        159500           22000              124007
  10  

,day,total_coupon_amount,total_remaining_cash,active_users,today_visitors,potential_visitors
0,1,168924000,2429921040,159500,22000,149648
1,2,161690800,2268230240,159500,22000,147644
2,3,156117600,2112112640,159500,22000,145222
3,4,149026800,1963085840,159500,22000,142484
4,5,143897200,1819188640,159500,22000,139453
5,6,138155600,1681033040,159500,22000,136138
6,7,131958800,1549074240,159500,22000,132439
7,8,126724400,1422349840,159500,22000,128444
8,9,121508800,1300841040,159500,22000,124007
9,10,116616400,1184224640,159500,22000,119324


In [ ]:
# 더 현실적인 시뮬레이션을 위한 추가 기능
def simulate_daily_visits_advanced(user_visit_gaps, daily_target, days=, seed=42):
    np.random.seed(seed)
    
    # 기본 방문 확률 계산
    visit_probs = 1 / user_visit_gaps
    visit_probs = visit_probs / visit_probs.sum()
    
    # 요일별 가중치 추가
    weekday_weights = {
        0: 1.2,  # 월요일
        5: 0.8,  # 토요일
        6: 0.7   # 일요일
    }
    
    daily_results = []
    for day in range(days):
        weekday = day % 7
        daily_weight = weekday_weights.get(weekday, 1.0)
        
        # 요일별 조정된 방문자 수
        adjusted_target = int(daily_target * daily_weight)
        
        # 방문자 샘플링
        visitors = np.random.choice(
            visit_probs.index,
            size=min(adjusted_target, len(visit_probs)),
            p=visit_probs,
            replace=False
        )
        
        daily_results.append({
            'day': day + 1,
            'weekday': weekday,
            'visitors': len(visitors),
            'unique_visitors': len(set(visitors))
        })
    
    return pd.DataFrame(daily_results)

In [ ]:
## 유저 방문 빈도 분포를 사전확률로 설정 시

import pandas as pd
import numpy as np

def find_optimal_coupon(cash, coupon_price):
   coupon_array = np.array(coupon_price)  
   tg_buy_idx = np.where((cash - coupon_array) >= 0, cash - coupon_array, np.inf).argmin()
   return coupon_array[tg_buy_idx], cash - coupon_array[tg_buy_idx]

def simulate_daily_purchases(user_data, days, coupon_price, min_coupon, daily_cap):
   results = []
   
   # 초기 활성 사용자 설정
   active_users = user_data.copy()
   active_users['cash'] = active_users['current_cash']  # 초기 잔액 설정
   
   for day in range(1, days + 1):
       # 당일 방문 가능한 사용자 (cash가 min_coupon 이상인 사용자)
       potential_users = active_users[active_users['cash'] >= min_coupon]
       
       if len(potential_users) > 0:
           # 일일 방문자 수를 daily_cap으로 제한
           n_visitors = min(len(potential_users), daily_cap)
           # 랜덤 샘플링으로 today_users 선택
           today_users = potential_users.sample(n=n_visitors).copy()
           
           # 쿠폰 구매 및 잔액 계산
           optimal_coupons = []
           left_cash = []
           
           for cash in today_users['cash']:
               coupon, remaining = find_optimal_coupon(cash, coupon_price)
               optimal_coupons.append(coupon)
               left_cash.append(remaining)
           
           # 사용자 잔액 업데이트
           today_users['cash'] = left_cash
           active_users.loc[today_users.index, 'cash'] = left_cash
           
           # 일일 결과 저장
           results.append({
               'day': day,
               'total_coupon_amount': sum(optimal_coupons),
               'total_remaining_cash': active_users['cash'].sum(),
               'active_users': len(active_users),
               'today_visitors': len(today_users),
               'potential_visitors': len(potential_users)
           })
       else:
           # 더 이상 방문 가능한 사용자가 없는 경우
           results.append({
               'day': day,
               'total_coupon_amount': 0,
               'total_remaining_cash': active_users['cash'].sum(),
               'active_users': len(active_users),
               'today_visitors': 0,
               'potential_visitors': 0
           })
           # 모든 사용자의 잔액이 min_coupon 미만이면 시뮬레이션 종료
           break
   
   return pd.DataFrame(results)

# 시뮬레이션 실행
days = 100
coupon_price = [400, 1200, 2000, 4000, 12000]
min_coupon = min(coupon_price)
daily_cap = 22000

df__ = df[['user_id', 'current_cash']].drop_duplicates()
results_df = simulate_daily_purchases(df__[['user_id', 'current_cash']], days, coupon_price, min_coupon, daily_cap)

# 결과 출력
print("\n일별 시뮬레이션 결과:")
print(results_df.to_string(index=False))

# 추가 분석
print(f"\n시뮬레이션 종료일: {len(results_df)}일")
print(f"최종 남은 총 잔액: {results_df.iloc[-1]['total_remaining_cash']:,.0f}원")
print(f"총 쿠폰 구매액: {results_df['total_coupon_amount'].sum():,.0f}원")

pd.set_option('display.max_rows', None) 
pd.DataFrame(results_df)



In [132]:
df__ = df[['user_id', 'current_cash']].drop_duplicates()

In [12]:
df[['user_id', 'current_cash']].drop_duplicates().current_cash.sum()


2246289196

퍼널

In [31]:
query = """with stack as 
  (select user_id, register_dt, sum(amt) as amt
  from
    (select m.wk_id as user_id, register_dt, amt 
    from ballosodeuk.dw.fact_shopby_reward r
    left join ballosodeuk.dw.dim_shopby_member m
      on r.member_no = m.member_no
    where 1=1
      and accumulation_status = "지급"
      and register_dt >= "2024-12-30"
      and reason_detail in ("쇼핑지원금 교환권 적립","쇼핑지원금 상품권 적립","쇼핑지원금 전환","쇼핑지원금 5,000원 교환 쿠폰" ))
  group by user_id, register_dt)

  ,click1 as 
  (select event_dttm, event_dt, user_id
  FROM `ballosodeuk.dw.fact_airbridge_event`, unnest(event_detail)
  where 1=1
    and event_dt between "2024-12-30" and "2025-01-09"
    and event_category in ('c__shopping_home__quick__btn (App)'))

,click2 as 
  (select event_dttm, event_dt, user_id
  FROM `ballosodeuk.dw.fact_airbridge_event`, unnest(event_detail)
  where 1=1
    and event_dt between "2024-12-30" and "2025-01-09"
    and event_category in ('c__all_income_s_money__to_exchange__btn (App)')
    )

,user_param as (
select distinct event_dt, user_id
from
  (select distinct event_dt, user_id from click1
  union all
  select distinct event_dt, user_id from click2
  ))

,exchange as 
  (
    select register_dttm, user_id, event, sum(amt) as amt
    from 
      (
        select register_dttm, wk_id as user_id, 
          case 
            when reason_detail in ('쇼핑지원금 교환권 적립', '쇼핑지원금 상품권 적립', '쇼핑지원금 5,000원 교환 쿠폰') then "교환권"
            else  "보상" end as event,
            amt
        from ballosodeuk.dw.fact_shopby_reward a
        left join `ballosodeuk.dw.dim_shopby_member` b on a.member_no = b.member_no
        where 1=1
          and accumulation_status = '지급'
          and register_dt >= "2024-12-30"
          and wk_id in (
            select user_id from user_param)
      )
    group by register_dttm, event, user_id
  )

,tg as 
  (select order_dttm,order_dt, order_no, member_no, first_pay_amt, first_sub_pay_amt, claim_status_type
  from
    (select order_dttm, order_dt, order_no, member_no, first_pay_amt, first_sub_pay_amt,  claim_status_type
    from ballosodeuk.dw.fact_shopby_order, unnest(product_option)
    where length(claim_status_type) < 1
    )
  group by order_dttm,order_dt, order_no, member_no, first_pay_amt, first_sub_pay_amt, claim_status_type)

,tgs as
  (select order_dttm as register_dttm, user_id, "구매" as event , first_pay_amt as amt
  from
    (
      select wk_id as user_id, min(order_dt) over (partition by a.member_no) as first_purchase, * except (member_no)
      from tg a
      left join ballosodeuk.dw.dim_shopby_member b on a.member_no = b.member_no
      where order_dt >= "2024-12-30"
    )
  -- where first_purchase >= "2024-12-30"
)

,merged as 
  (select * from exchange
  union all 
  select * from tgs)

,funnel as 
  (select register_dttm, user_id
    ,funnel_1
    ,case when funnel_1 = "교환권" then funnel_1_dt end as funnel_1_dt
    ,case when funnel_2 = "구매" and funnel_2_dt > funnel_1_dt then funnel_2 end as funnel_2
    ,case when funnel_2 = "구매" and funnel_2_dt > funnel_1_dt then funnel_2_dt end as funnel_2_dt
    ,case when 
      funnel_2 = "구매" 
      and funnel_2_dt > funnel_1_dt
      and funnel_3 = "보상" 
      and funnel_3_dt > funnel_2_dt
      then funnel_3 end as funnel_3
    ,case when 
      funnel_2 = "구매" 
      and funnel_2_dt > funnel_1_dt
      and funnel_3 = "보상" 
      and funnel_3_dt > funnel_2_dt
      then funnel_3_dt end as funnel_3_dt
  from
    (select register_dttm,user_id, event as funnel_1, register_dttm as funnel_1_dt
      ,lead(event) over (partition by user_id order by register_dttm) as funnel_2
      ,lead(register_dttm) over (partition by user_id order by register_dttm) as funnel_2_dt
      ,lead(event, 2) over (partition by user_id order by register_dttm) as funnel_3
      ,lead(register_dttm, 2) over (partition by user_id order by register_dttm) as funnel_3_dt
    from merged)
  where 1=1
    and funnel_1 = "교환권")

select * from funnel"""

In [32]:
ff = bq.query(query).to_dataframe()

c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


In [4]:
df

,user_id,event_dt,current_cash
10,000087ef-6929-4532-85be-33661b2082d5,2025-01-06,25905
11,0000d355-7050-4a3e-90d1-2aa0b1198aba,2024-12-30,8636
12,0000d355-7050-4a3e-90d1-2aa0b1198aba,2024-12-31,8636
13,00022901-aed3-4327-bb0a-29bbdc978dfa,2024-12-30,18658
14,000256cd-ebbd-46ac-94e9-93a2b1ace57d,2025-01-05,7283
...,...,...,...
349932,fffff0da-172e-421f-87be-12324d714cbd,2025-01-04,14865
349933,fffff0da-172e-421f-87be-12324d714cbd,2025-01-06,14865
349934,fffff0da-172e-421f-87be-12324d714cbd,2025-01-07,14865
349935,fffff0da-172e-421f-87be-12324d714cbd,2025-01-08,14865


In [23]:
df = df.groupby('user_id').agg({'current_cash':'first'})
df = df.reset_index()

In [27]:
bins = [
    0,
    200,
    600,
    1000,
    2000,
    6000,
    10000,
    20000,
    float('inf')
]

labels = [
    "0 ~ 200",
    "200 ~ 600", 
    "600 ~ 1,000",
    "1,000 ~ 2,000",
    "2,000 ~ 6,000",
    "6,000 ~ 10,000",
    "10,000 ~ 20,000",
    "20,000 ~"
]

df['cash_bin'] = pd.cut(df['current_cash'], bins=bins, labels=labels)
df_ = df.groupby('cash_bin').agg({'user_id': lambda x: len(set(x)), 'current_cash': 'sum'})
df_ = df_.reindex(labels)
df_


,user_id,current_cash
cash_bin,,
0 ~ 200,2914,234471
200 ~ 600,2011,788474
"600 ~ 1,000",1765,1417268
"1,000 ~ 2,000",4942,7456447
"2,000 ~ 6,000",21810,86727898
"6,000 ~ 10,000",17476,137782372
"10,000 ~ 20,000",28387,411599465
"20,000 ~",44824,1596926546


In [25]:
df_.current_cash.sum()

2242932941